In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math 
from collections import defaultdict

In [6]:
complete_data = pd.read_csv("datathon_2024_dataset.csv")

In [7]:
complete_data = complete_data[['game_date','home_team','away_team','city']]

In [8]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
geolocator = Nominatim(user_agent="MyGeocoder/1.0.0 (kig3@rice.edu)",timeout=100)

location_dataset = []
unique_values = complete_data['city'].unique()

for city in unique_values:
    loc = geolocator.geocode(city)
    
    # Check if location is found before accessing latitude and longitude
    if loc:
        location_data = {
            'city': city,
            'latitude': loc.latitude,
            'longitude': loc.longitude
        }
        location_dataset.append(location_data)
location_dataset = pd.DataFrame(location_dataset)

In [9]:
final_data = []

In [10]:
complete_data['game_date'] = complete_data['game_date'].astype(str)
for i in range(2000,2024):
    year = str(i)
    data = complete_data[complete_data['game_date'].str[:4] == year]
    data['game_date'] = pd.to_datetime(data['game_date'], format='%Y%m%d')
    distance_map = defaultdict(list)
    for index, row in data.iterrows():
        curr_location = location_dataset.loc[location_dataset['city'] == row['city']].iloc[0]
        if len(distance_map[row['home_team']]) == 0:
            data.at[index, 'distance_from_last_location_home_team'] = 0
        else:
            data.at[index, 'distance_from_last_location_home_team'] = geodesic((curr_location['latitude'], curr_location['longitude']), 
                                                                           (distance_map[row['home_team']][0], distance_map[row['home_team']][1])).km
        distance_map[row['home_team']].append(curr_location['latitude'])  
        distance_map[row['home_team']].append(curr_location['longitude'])
    
        if len(distance_map[row['away_team']]) == 0:
            data.at[index, 'distance_from_last_location_away_team'] = 0
        else:
            data.at[index, 'distance_from_last_location_away_team'] = geodesic((curr_location['latitude'], curr_location['longitude']), 
                                                                           (distance_map[row['away_team']][0], distance_map[row['away_team']][1])).km
        distance_map[row['away_team']].append(curr_location['latitude'])  
        distance_map[row['away_team']].append(curr_location['longitude'])
    final_data.append(data)
final_df = pd.concat(final_data, ignore_index=True) 

C:\Users\kusha\AppData\Local\Temp\ipykernel_23068\3608874014.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['game_date'] = pd.to_datetime(data['game_date'], format='%Y%m%d')
C:\Users\kusha\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\kusha\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try 

In [11]:
print(final_df)

       game_date home_team away_team         city  \
0     2000-03-29       NYN       CHN        Tokyo   
1     2000-03-30       CHN       NYN        Tokyo   
2     2000-04-03       ATL       COL      Atlanta   
3     2000-04-03       CIN       MIL   Cincinnati   
4     2000-04-03       FLO       SFN        Miami   
...          ...       ...       ...          ...   
56770 2023-10-01       CHA       SDN      Chicago   
56771 2023-10-01       DET       CLE      Detroit   
56772 2023-10-01       KCA       NYA  Kansas City   
56773 2023-10-01       SEA       TEX      Seattle   
56774 2023-10-01       TOR       TBA      Toronto   

       distance_from_last_location_home_team  \
0                                   0.000000   
1                                   0.000000   
2                                   0.000000   
3                                   0.000000   
4                                   0.000000   
...                                      ...   
56770                      

In [12]:
final_df.to_csv('distance_covered_since_last_game.csv')